In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Import the pandas library to read our dataset 
import pandas as pd
# Get the train/test split package from sklearn for preparing our dataset to # train and test the model with 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

import nltk 
import random 
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords')  
nltk.download('wordnet') 

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
import csv
import urllib.request as urllib2

import matplotlib.pyplot as plt

import pandas as pd
# Importing data using url
url = 'https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
response = urllib2.urlopen(url)

# reading data and converting into dataframe
df = pd.read_csv(response,delimiter='\t',encoding='utf-8')
df.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,48716,2379,wastes an exceptionally good idea .,1
1,106634,5631,some of the more serious-minded concerns of ot...,2
2,146976,7999,does n't hurt anyone and works for its partici...,3
3,87430,4535,right choice,3
4,45764,2225,The Tuxedo was n't just bad ;,0


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(df ['Phrase'], df ['Sentiment'], test_size=0.3, random_state=2003)
documents=[]
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())
for i in range(len(X_train)):
  documents.append([list(word_tokenize(X_train[i])), Y_train[i]]) 

X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  documents.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

documents[0]

[['perfect', 'starting', 'point'], 3]

In [0]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()" 
remove_stopwords = True
useStemming = False
useLemma = False
removePuncs = True


In [7]:
for l in range(len(documents)):                    
  label = documents[l][1]                          
  tmpReview = []                                   
  for w in documents[l][0]:                        
    newWord = w                                    
    if remove_stopwords and (w in stopwords_en): 
      continue                                     
    if removePuncs and (w in punctuations):        
      continue                                     
    if useStemming: 
      newWord = lancaster.stem(newWord)  
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    tmpReview.append(newWord)                      
  documents[l] = (tmpReview, label)              
  documents[l] = (' '.join(tmpReview), label) 

print(documents[0])

('perfect starting point', 3)


In [8]:
df = pd.DataFrame(documents, columns=['text', 'sentiment']) 
df.head()

,text,sentiment
0,perfect starting point,3
1,A weird,2
2,Stealing Harvard,2
3,Robin Williams,2
4,As plain pedestrian catsup,0


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],  df['sentiment'], test_size=0.3, random_state=2003)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

vectorizer = TfidfVectorizer(max_features = 2000) 
X = vectorizer.fit_transform(df["text"]) 
Y = df['sentiment'] 
 
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test


Using TensorFlow backend.


In [11]:
Y_test

13510     2
61932     2
82549     3
137718    3
121990    3
         ..
94224     4
135456    4
154729    3
23031     2
57870     1
Name: sentiment, Length: 46818, dtype: int64

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K

In [0]:
batch_size = 64
num_classes = 5
epochs = 12

In [15]:
X_train.shape

(109242, 2000)

In [0]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)


In [17]:
Y_test

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [18]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3,
                 activation='relu',
                 input_shape=(2000,1)))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(rate = 0.50))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [0]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=25)
# _, accuracy = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/25





109242/109242 [==============================] - 1863s 17ms/step - loss: 1.1464 - acc: 0.5541
Epoch 2/25
109242/109242 [==============================] - 1867s 17ms/step - loss: 1.0611 - acc: 0.5864
Epoch 3/25
109242/109242 [==============================] - 1862s 17ms/step - loss: 1.0357 - acc: 0.6000
Epoch 4/25
109242/109242 [==============================] - 1879s 17ms/step - loss: 1.0262 - acc: 0.6059
Epoch 5/25
109242/109242 [==============================] - 1869s 17ms/step - loss: 1.0207 - acc: 0.6075
Epoch 6/25
109242/109242 [==============================] - 1895s 17ms/step - loss: 1.0154 - acc: 0.6092
Epoch 7/25
109242/109242 [==============================] - 1921s 18ms/step - loss: 1.0112 - acc: 0.6110
Epoch 8/25
109242/109242 [==============================] - 1920s 18ms/step - loss: 1.0077 - acc: 0.6132
Epoch 9/25
 63744/109242 [================>.............] 